## Step 1: Install Ollama (if not installed)
   Ollama is the easiest way to run DeepSeek on macOS.
1. Download and install Ollama: <br>
    ```curl -fsSL https://ollama.com/install.sh | sh```
2. Verify installation: <br>
    ```ollama version```

## Step 2: Pull the DeepSeek Model
   Ollama supports DeepSeek Coder and DeepSeek Chat models.
*  For DeepSeek Coder (code generation & completion): <br>
        ```ollama pull deepseek-coder```
*  For DeepSeek Chat (general AI chat): <br>
        ```ollama pull deepseek-chat```

##  Step 3: Run the DeepSeek Model
    Once downloaded, you can run DeepSeek locally:

*   Run an interactive chat session: <br>
        ```ollama run deepseek-chat```
*   Run a command-line query: <br>
        ```ollama run deepseek-chat "Explain LangChain with an example"```

##  Step 4: Use DeepSeek in Python (Optional)
*    If you want to use DeepSeek in a Python script with Ollama API, install ollama in Python: <br>
        ```pip install ollama```
*   Then, call the model:

In [ ]:
import ollama

response = ollama.chat(model='deepseek-llm:7b-chat', messages=[{'role': 'user', 'content': 'What is LangChain?'}])
print(response['message']['content'])


Let me know if you need further customization, like using DeepSeek in LangChain! 🚀

You can achieve this using LangChain with Ollama and a ReAct agent. Here’s how:

#### **Approach**
1. ***Upload & Parse PDF*** – Use ```PyMuPDF``` or ```pdfplumber``` to extract text.
2. ***Memory Handling*** – Store extracted text in a retriever (e.g., FAISS, ChromaDB).
3. ***ReAct Agent*** – Create a LangChain agent with ***Ollama (DeepSeek or Mistral)*** as the LLM.
4. ***Query Answering*** – Use the extracted text as context for responses.

------------------------

#### **Implementation Steps**
1. **Install Dependencies** <br>
        ```pip install langchain langchain-community ollama pypdf```
2. **PDF Parsing & Retrieval**

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Load the PDF
loader = PyPDFLoader("English.pdf")
docs = loader.load()

# Create vector store
vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

#### **3. Set Up Ollama ReAct Agent**

In [ ]:
from langchain.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# Use Ollama with DeepSeek
llm = Ollama(model="deepseek-chat")

# Define the retriever as a LangChain Tool
retriever_tool = Tool(
    name="PDF Retriever",
    func=lambda query: retriever.get_relevant_documents(query),
    description="Retrieves relevant information from the uploaded PDF."
)

# Initialize ReAct Agent
agent = initialize_agent(
    tools=[retriever_tool],
    llm=llm,
    agent=AgentType.REACT_DOCSTORE,
    verbose=True
)

# Run a query
response = agent.run("What is discussed in the first section of the PDF?")
print(response)

### **Enhancements**
* **Use ChromaDB for faster retrieval**
* **Use Mistral-7B if DeepSeek lacks context depth**
* **Add ConversationalBufferMemory for multi-turn chats** <br>
Would you like a Streamlit or FastAPI interface for this? 🚀
----------

In [71]:
import json
import os
from langchain.agents import Tool, AgentExecutor
from langchain.agents.react import ReActAgent, ReActOutputParser
from langchain.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

ImportError: cannot import name 'ReActAgent' from 'langchain.agents.react' (/opt/anaconda3/lib/python3.12/site-packages/langchain/agents/react/__init__.py)

In [45]:
# Custom ReAct Output Parser to ensure JSON formatting
class StrictJSONOutputParser(ReActOutputParser):
    def parse(self, text):
        try:
            # Attempt to parse the output into JSON
            return json.loads(text)
        except json.JSONDecodeError:
            # If JSON parsing fails, return a structured JSON with an error explanation
            return {
                "answer": "",
                "explanation": f"Output parsing error: Invalid JSON output received - {text}"
            }

In [ ]:
# Sample function for the tool (replace with actual function)
def pdf_retriever_function(input_data):
    # This is a placeholder function, implement as needed for your use case
    return "Relevant data from the PDF document."

In [ ]:
# Define tools (if needed)
tools = [
    Tool(
        name="PDF Retriever",
        func=pdf_retriever_function,  # Replace with actual function
        description="Retrieve relevant information from the uploaded PDF"
    )
]

In [ ]:
# Define the system prompt to enforce strict JSON output
system_prompt = """You are an AI assistant using the ReAct framework.
When answering a question, always return a strict JSON response in the following format:
{
  "answer": "concise response to the question",
  "explanation": "optional extra details if needed, else empty string"
}
Make sure that the response is **only** valid JSON. Do not output any plain text or extra characters. Any invalid response should return an error message in the explanation field, explaining why the response is not valid JSON.
"""

In [ ]:
# Initialize the Ollama model (replace with your model identifier)
model = Ollama(model="deepseek-llm:7b-chat")

# Define the ReAct agent with the custom output parser
output_parser = StrictJSONOutputParser()
agent = ReActAgent(
    tools=tools,
    output_parser=output_parser,
    llm=model,  # Use the Ollama model
    system_message=system_prompt
)

In [ ]:
# Create an agent executor
executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
# Function to load a PDF and create a vector store
def load_pdf_and_create_vector_store(pdf_path):
    # Load the PDF document using PyPDFLoader
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    # Create embeddings (you can replace OpenAIEmbeddings with any other embeddings you are using)
    embeddings = OpenAIEmbeddings()

    # Create FAISS vector store
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore

In [ ]:
# Function to query the agent and ensure JSON response
def query_agent(question):
    # Run the agent with the provided question
    response = executor.run({"input": question})
    
    # Log the raw response for debugging
    print("Raw response from agent:", response)
    
    # Ensure the response is valid JSON (it will be after parsing)
    return response


In [ ]:
# Path to the PDF file
pdf_path = "English.pdf"  # Replace with your PDF path

# Load the PDF and create the vector store
vectorstore = load_pdf_and_create_vector_store(pdf_path)


In [ ]:
# Example query
query = "What is the role of counselling in preventing drug addiction?"
response_json = query_agent(query)

# Print the structured JSON response
print("Structured JSON response:", json.dumps(response_json, indent=2))